## Bag of Words

taking the data and preprocessing to generate a bag of words for each ASIN

In [18]:
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer

from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
# this one takes a lil while so be careful
df = pd.read_json('data/Sports_and_Outdoors_Reviews_training.json', lines=True)

# take out reviews without reviewText and sort them by ASIN
df = df[df['reviewText'].notna()]
df = df[df['asin'].notna()]
df_sorted = df.sort_values(by='asin')
df_sorted

In [10]:
# showing breakdown of review ratings
df_sorted.overall.value_counts()

5    1544003
4     397940
3     168761
1      89173
2      81544
Name: overall, dtype: int64

In [35]:
# generating document term matrix (DTM) 
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english', ngram_range = (1,3), tokenizer=token.tokenize)
text_counts = cv.fit_transform(df_sorted['reviewText'])

all the comments below are a skeleton for eventually doing k-fold cross validation

In [37]:
# splitting data for testing and training to see how well model performs

# from sklearn.cross_validation import StratifiedShuffleSplit
# sss = StratifiedShuffleSplit(y, n_iter=1, test_size=0.5, random_state=0)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, df_sorted['overall'], test_size = 0.25, random_state = 5)
# for train_idx, test_idx in sss:
#     X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]
#     svc.fit(X_train, y_train)
#     y_pred = svc.predict(X_test)
#     print(f1_score(y_test, y_pred, average="macro"))
#     print(precision_score(y_test, y_pred, average="macro"))
#     print(recall_score(y_test, y_pred, average="macro"))    

hey chris check this out

In [38]:
# fitting the model
CNB = ComplementNB()
CNB.fit(X_train, Y_train)

ComplementNB()

In [39]:
# checking accuracy
from sklearn import metrics
predicted = CNB.predict(X_test)
f1_score = metrics.f1_score(Y_test, predicted, average='weighted')
# auc_score = metrics.roc_auc_score(Y_test, predicted, average='weighted')
print('F1: ' + str('{:04.2f}'.format(f1_score*100)) + '%')
# print('AUC: ' + str('{:04.2f}'.format(auc_score*100)) + '%')

F1: 69.33%
